# Distance modulus $\mu$ from SNANA-SALT2 fit

The distance modulus is determined from the SNANA-SALT2 fit and the global parameters reported in Table 6 of Scolnic et al. 2017 (https://arxiv.org/abs/1710.00845).

This script outputs:
    - a text file (with my format) of the distance moduli to create the Hubble diagram with 11.ipynb
    - a simple Hubble diagram plot.

# USER

In [26]:
# This corresponds to the SNANA version used 
# during the computations. This is needed because different versions
# of SNANA produce different format of the output fitres files.
# Options: (v10_58g, v10_35g, v10_45j)
FitresFormat = 'v10_58g'   

NotebookName = 'SNANA_SALT2_DistanceMu.ipynb'

#--------------

sigma_vPec = 150  # 150 km/s 

cc = 299792.458  # Speed of light (km/s)
MoFix = 19.36 # Absolute magnitude at T_Bmax. This is just a nuisance parameter
HoFix = 73.24  # 70 = SNANA default value
OmMFix = 0.28 # Omega_Matter # 0.3 = SNANA default value
OmLFix = 0.72 # Omega_Lambda # 0.7 = SNANA default value
wFix = -1 # Dark energy EoS # -1  = SNANA default value


---------

# Automatic

## Some definitions

In [19]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.integrate import quad as intquad

5+4

9

In [20]:
# OK
""" 
# Distance modulus from modified Tripp formula as in Scolnic+17

def muSALT2_func(mB, M, alpha, x1, beta, c1, gamma, tau, Mass, MassStep, Delta_mB, Delta_c, Delta_x1): 
    
    DeltaM = gamma/(1 + np.exp(-(Mass-MassStep)/tau))
    DeltaB = Delta_mB - beta*Delta_c + alpha*Delta_x1
    distanceMu = mB - M + alpha*x1 - beta*c1 + DeltaM + DeltaB
    
    return distanceMu, DeltaM, DeltaB

# Test from table 16.
print '#', muSALT2_func(21.86, -18, 0.156, -2.3, 3.02, 0.05, 0.053, 0.001, 10.90, 10.13, 0.04, 0.025, 0.7)
# (39.476899999999993, 0.052999999999999999, 0.07369999999999999)
"""
0

0

In [21]:
# OK
"""
# Distance modulus from modified Tripp formula as in Rest+14

def muSALT2_Rest14_func(mB, M, alpha, x1, beta, c1): 
    distanceMu = mB - M + alpha*x1 - beta*c1
    return distanceMu

#-------------------------------------------

# Test

Mfix = -19.35
alphaFix = 0.147
betaFix = 3.13

print '# sn1990O (JRK07)', muSALT2_Rest14_func(15.993, Mfix, alphaFix, 0.476, betaFix, -0.054)
# sn1990O (JRK07) 35.581992

print '# sn2005el (CfA3)', muSALT2_Rest14_func(14.671, Mfix, alphaFix, -1.242, betaFix, -0.070)
# sn2005el (CfA3) 34.057526
"""
0

0

In [22]:
# OK
# David Jones codes:
    
def salt2mu(z=None, x0=None, c=None, cerr=None, x1=None, x1err=None,
             mb=None, mberr=None, cov_x1_x0=None, cov_c_x0=None, cov_x1_c=None, 
            alpha=None, beta=None, sigint=None, Mo=None, sigma_vPec=None):

    # The distance modulus mean value:
    mu_out = mb + x1*alpha - beta*c + MoFix     
     
    #  Computing the distance-modulus uncertainty (Tripp formula):
    sf = -2.5/(x0*np.log(10.0))
    cov_mb_c = cov_c_x0*sf
    cov_mb_x1 = cov_x1_x0*sf
    
    # Propagation of uncertainty from Tripp formula and accountig for correlations:
    sigma2_mu_Tripp = (mberr**2. + (alpha**2.)*(x1err**2.) + (beta**2.)*(cerr**2.) + \
                         2.0*alpha*cov_mb_x1 - 2.0*beta*cov_mb_c - \
                         2.0*alpha*beta*cov_x1_c )

    # Uncertainty in distance modulus due to peculiar velocity uncertainty:
    zerr = (sigma_vPec/cc)*5.0/np.log(10)*(1.0+z)/(z*(1.0+z/2.0))
    
    # Total distance-modulus uncertainty:
    muerr_out = np.sqrt(sigma2_mu_Tripp + zerr**2. + (0.055**2.)*(z**2.) + sigint**2. )
    
    return (mu_out, muerr_out, sigma2_mu_Tripp)

# print '# Test' 
alpha_test = 0.147 
beta_test =  3.00
sigmaInt_test = 0.11

# print '# 2006le (PS1s_CFA3_KEPLERCAM_RS14):', \
# salt2mu(0.0172, 0.26064E-01, -0.0669,  0.0640,  0.8947,  0.0506, 
#         14.5949,  0.2197,   0.895E-04,  -0.349E-03,  -0.104E-02, 
#         alpha_test, beta_test, sigmaInt_test, MoFix, sigma_vPec)  

# print '# 2006lf (PS1s_CFA3_KEPLERCAM_RS14):', \
# salt2mu(0.0124, 0.78991E-01, -0.2086,  0.1013, -1.3905, 0.0674, 
#         13.3911,  0.3646, 0.228E-03,  -0.376E-02,  -0.721E-03, 
#         alpha_test, beta_test, sigmaInt_test, MoFix, sigma_vPec)  
        

In [23]:
# Inverse of the dimensionless Hubble parameter
def InvEHubblePar(z, OmM, wde):
    "Dimensionless Hubble parameter"
    InvEHubbleParInt = 1.0/(np.sqrt(OmM*(1.0+z)**3.0 + (1.0-OmM)*(1.+z)**(3.*(1.+wde))))
    return InvEHubbleParInt

# ---- The luminosity distance ----
def LumDistanceVec(z, OmM, wde, Ho):
    "Luminosity distance"
    LumDistanceVecInt = 0.
    LumDistanceVecInt = cc*(1.+z)*intquad(InvEHubblePar, 0., z, args=(OmM, wde))[0]/Ho 
    return LumDistanceVecInt

# ---- Distance modulus scalar ----
def DistanceMu(z, OmM, wde, Ho):
    "Distance modulus"     
    DistanceMuInt = 5.0*np.log10(LumDistanceVec(z, OmM, wde, Ho)) + 25.0
    return DistanceMuInt

# ---- Distance modulus Vector ----
def DistanceMuVector(z, OmM, wde, Ho):
    "Distance modulus"     
    DistanceMuInt= []
    for i in range(len(z)):
        DistanceMuInt += [5.0*np.log10(LumDistanceVec(z[i], OmM, wde, Ho)) + 25.0] 
    return DistanceMuInt

#--------------------------------------------------

ztest1 = 0.01

print '# Checking that the functions work well:', DistanceMu(ztest1, OmMFix, wFix, HoFix)
# Checking that the functions work well: 33.1753183809

# Checking that the functions work well: 33.07739265766354


In [24]:
# sigma^2_mu from the peculiar velocity uncertainty
# This function is used to determine in the sections "Intrinsic dispersion" and "Optical RMS", to
# determine the intrinsic dispersion.

def sigma2_pec(zcmb, err_zcmb, vpec):
    sigma2_pecInt = ((5/(zcmb*np.log(10)))*np.sqrt((vpec/cc)**2 + err_zcmb**2))**2
    return sigma2_pecInt

# Test
sigma2_pec(0.0109942726, 0.0010420420, 150)
# 0.052125037354329877

0.05212503735432988

### Cepheid distances

In [9]:
# OK but not used
""" 
DirSNeWithCepheid = '/Users/arturo/Dropbox/Research/SoftwareResearch/Snoopy/AndyLCComp/MyNotesAbout/'

ListSNeCepheid = np.genfromtxt(DirSNeWithCepheid+'SNeWithCepheidDistances.txt', dtype=['S10', 
                                                float,float,float,float,float,float])
# ListSNeCepheid = np.genfromtxt(DirSNeWithCepheid+'SNeWithCepheidDistances_hack.txt', dtype=['S10',
#                                                 float,float,float,float,float,float]) 

ListSNeCepheid['f0']
"""
0

0

#### Get the name of this ipython notebook
To print it in the output text files as reference

In [10]:
%%javascript
var kernel = IPython.notebook.kernel;
var thename = window.document.getElementById("notebook_name").innerHTML;
var command = "NotebookName = " + "'"+thename+".ipynb"+"'";
kernel.execute(command);

<IPython.core.display.Javascript object>

In [11]:
print '#', (NotebookName)
# Update_zcmb_in_SNANA_datafiles_v1_0.ipynb

# SNANA_SALT2_DistanceMu.ipynb


In [12]:
# Get the current date and time
import datetime 

# Read the time and date now
now = datetime.datetime.now()

---------

# Compute $(\mu, \sigma_{\mu})$

### User

In [13]:
# USER

"""
# DES

DirSALT2Fit = '/Users/arturo/Dropbox/Research/Articulos/12_RAISINs/Data/\
RAISIN_2/Data/DES/2017_11_21/Photometry/fit_SomeLinesCommented_ok/'

sample = 'snana_salt2_fit' # Folder name
FitresFile = 'des.fitres'
SampleFlag = '1'   # 1 =CfA, 2=CSP, 3=Others


# PS1 + DES 
DirSALT2Fit = '/Users/arturo/Dropbox/Research/Articulos/12_RAISINs/Data/raisin12/'

sample = 'hubblediagram' # Folder name
FitresFile = 'ps1_des_Notes_.fitres'
SampleFlag = '1'   # 1 =CfA, 2=CSP, 3=Others

#-----------------------------

# The distance modulus is determined from the SNANA-SALT2 fit and the global 
# parameters reported in Table 6 of Scolnic et al. 2017 
# (https://arxiv.org/abs/1710.00845).

# Global parameters in the Tripp formula for: high-z.

alpha = 0.165 
beta =  2.93
sigmaInt = 0.09
""" 

0

0

In [15]:
# USER

#   LOW_Z SAMPLE

DirSALT2Fit = '/Users/arturo/Dropbox/Research/\
SoftwareResearch/SNANA/Odyssey/home_snana/lowz/\
3_GaussianProcessSubsample/cutoffs_no/'

FitresFile = 'GP_subsample_.fitres'
SampleFlag = '1'   # 1 =CfA, 2=CSP, 3=Others


# sample = 'GaussianProcessSubsample' 

#---------------------

"""
sample = 'JLA2014_CfAIII_KEPLERCAM' # Folder name
FitresFile = 'JLA2014_CfAIII_KEPLERCAM.fitres'
SampleFlag = '1'   # 1 =CfA, 2=CSP, 3=Others
"""

"""
sample = 'JLA2014_CSP' # Folder name
FitresFile = 'JLA2014_CSP.fitres'
SampleFlag = '2'   # 1 =CfA, 2=CSP, 3=Others
"""

"""
sample = 'JLA2014_LOWZ_LANDOLT' # Folder name
FitresFile = 'JLA2014_LOWZ_LANDOLT.fitres'
SampleFlag = '2'   # 1 =CfA, 2=CSP, 3=Others
"""

"""
sample = 'LOWZ_FromAndyFriedman' # Folder name
FitresFile = 'LOWZ_FromAndyFriedman.fitres'
SampleFlag = '1'   # 1 =CfA, 2=CSP, 3=Others
"""

"""
sample = 'LOWZ_JRK07' # Folder name
FitresFile = 'LOWZ_JRK07.fitres'
SampleFlag = '2'   # 1 =CfA, 2=CSP, 3=Others
"""

"""
sample = 'PS1s_CFA3_4SHOOTER2_RS14' # Folder name
FitresFile = 'PS1s_CFA3_4SHOOTER2_RS14.fitres'
SampleFlag = '1'   # 1 =CfA, 2=CSP, 3=Others
"""

"""
sample = 'PS1s_CFA3_KEPLERCAM_RS14' # Folder name
FitresFile = 'PS1s_CFA3_KEPLERCAM_RS14.fitres'
SampleFlag = '1'   # 1 =CfA, 2=CSP, 3=Others
"""

"""
sample = 'PS1s_CFA4_p1_RS14' # Folder name
FitresFile = 'PS1s_CFA4_p1_RS14.fitres'
SampleFlag = '1'   # 1 =CfA, 2=CSP, 3=Others
"""

"""
sample = 'PS1s_CFA4_p2_RS14' # Folder name
FitresFile = 'PS1s_CFA4_p2_RS14.fitres'
SampleFlag = '1'   # 1 =CfA, 2=CSP, 3=Others
"""

"""
sample = 'PS1s_CSPDR2' # Folder name
FitresFile = 'PS1s_CSPDR2.fitres'
SampleFlag = '2'   # 1 =CfA, 2=CSP, 3=Others
"""


#-----------------------------

# The distance modulus is determined from the SNANA-SALT2 fit and the global 
# parameters reported in Table 6 of Scolnic et al. 2017 
# (https://arxiv.org/abs/1710.00845).

# Global parameters in the Tripp formula for: low-z.

alpha = 0.147 
beta =  3.00
sigmaInt = 0.11


#-----------------------------
0

0

### Automatic

In [28]:

#  SNANA_v10_35g fitres file:
if FitresFormat == 'v10_35g': 
    try: 
        snanaSalt2Fit_np =np.genfromtxt(DirSALT2Fit+FitresFile[:-7]+'Notes_.fitres', 
                                        skip_header=2, comments='#',
                                        dtype=['S3', 'S15',
                                              float, float, float, float, float, float, float,
                                              float, float, float, float, float, float, float, float,
                                              float, float, float, float, float, float, float, float] )
    except:
        snanaSalt2Fit_np =np.genfromtxt(DirSALT2Fit+FitresFile, 
                                        skip_header=2, comments='#',
                                        dtype=['S3', 'S15',
                                              float, float, float, float, float, float, float,
                                              float, float, float, float, float, float, float, float,
                                              float, float, float, float, float, float, float, float] ) 


#  SNANA_v10_58g fitres file:
elif FitresFormat == 'v10_58g':
    snanaSalt2Fit_np = np.genfromtxt(DirSALT2Fit+FitresFile, skip_header=8,
                        dtype=['S3', 'S15', float, float, 'S4',
                              float, float, float, float, float,
                              float, float, float, float, float,
                              float, float, float, float, float,
                              float, float, float, float, float,
                              float, float, float, float, float,
                              float, float, float, float, float]) 

    
#  David Jones fitres file:
elif FitresFormat == 'v10_45j': 
    snanaSalt2Fit_np =np.genfromtxt(DirSALT2Fit+FitresFile, 
                    skip_header=2, comments='#',
                    dtype=['S3', 'S15',float, float, 'S4', 
                          float, float, float, float, float, float, float, float, float, float,
                          float, float, float, float, float, float, float, float, float, float,
                          float, float, float, float, float, float, float, float, float, float] ) 


#----------------

DirSaveOutput = DirSALT2Fit

print '# %s SNe in this file.'%(len(snanaSalt2Fit_np))
# 55 SNe in this file.

# 56 SNe in this file.


In [29]:
DirSaveOutput

'/Users/arturo/Dropbox/Research/SoftwareResearch/SNANA/Odyssey/home_snana/lowz/3_GaussianProcessSubsample/cutoffs_no/'

In [30]:
# Computing the distance modulus

# Define some variable arrays:


# Reading my own fitres files:
if FitresFormat == 'v10_35g': 
    z_np =  snanaSalt2Fit_np['f2'] # z_CMB
    zerr_np = snanaSalt2Fit_np['f3'] # Not needed to compute the distance modulus.
    x0_np = snanaSalt2Fit_np['f4']
    c_np = snanaSalt2Fit_np['f6']
    cerr_np = snanaSalt2Fit_np['f7']
    x1_np = snanaSalt2Fit_np['f8']
    x1err_np = snanaSalt2Fit_np['f9']
    mb_np =  snanaSalt2Fit_np['f12']
    mberr_np =  snanaSalt2Fit_np['f13']
    cov_x1_c_np =  snanaSalt2Fit_np['f16']
    cov_x1_x0_np =  snanaSalt2Fit_np['f14']
    cov_c_x0_np =  snanaSalt2Fit_np['f15']
    chi2_np = snanaSalt2Fit_np['f17'] # Not needed to compute the distance modulus.
    Ndof_np = snanaSalt2Fit_np['f18']  # Not needed to compute the distance modulus.

# Reading my own fitres files:
elif FitresFormat == 'v10_58g': 
    z_np =  snanaSalt2Fit_np['f10'] # z_CMB
    zerr_np = snanaSalt2Fit_np['f11'] # Not needed to compute the distance modulus.
    x0_np = snanaSalt2Fit_np['f27']
    c_np = snanaSalt2Fit_np['f23']
    cerr_np = snanaSalt2Fit_np['f24']
    x1_np = snanaSalt2Fit_np['f21']
    x1err_np = snanaSalt2Fit_np['f22']
    mb_np =  snanaSalt2Fit_np['f25']
    mberr_np =  snanaSalt2Fit_np['f26']
    cov_x1_c_np =  snanaSalt2Fit_np['f29']
    cov_x1_x0_np =  snanaSalt2Fit_np['f30']
    cov_c_x0_np =  snanaSalt2Fit_np['f31']
    chi2_np = snanaSalt2Fit_np['f33'] # Not needed to compute the distance modulus. 
    Ndof_np = snanaSalt2Fit_np['f32']  # Not needed to compute the distance modulus.
    
#   David Jones fitres file:
elif FitresFormat == 'v10_45j': 
    z_np =  snanaSalt2Fit_np['f6'] # z_CMB
    zerr_np = snanaSalt2Fit_np['f7'] # Not needed to compute the distance modulus.
    x0_np = snanaSalt2Fit_np['f25']
    c_np = snanaSalt2Fit_np['f21']
    cerr_np = snanaSalt2Fit_np['f22']
    x1_np = snanaSalt2Fit_np['f19']
    x1err_np = snanaSalt2Fit_np['f20']
    mb_np =  snanaSalt2Fit_np['f23']
    mberr_np =  snanaSalt2Fit_np['f24']
    cov_x1_c_np =  snanaSalt2Fit_np['f27']
    cov_x1_x0_np =  snanaSalt2Fit_np['f28']
    cov_c_x0_np =  snanaSalt2Fit_np['f29']
    chi2_np = snanaSalt2Fit_np['f31'] # Not needed to compute the distance modulus. 
    Ndof_np = snanaSalt2Fit_np['f30']  # Not needed to compute the distance modulus.

#------------------

# Computing the distance modulus:
mu_SALT2 = salt2mu(z_np, x0_np, c_np, cerr_np, x1_np, x1err_np, 
                   mb_np, mberr_np, cov_x1_x0_np, cov_c_x0_np, cov_x1_c_np, 
                  alpha, beta, sigmaInt, MoFix, sigma_vPec)

In [31]:
# Inspect for 'nan' values in the total distance-modulus uncertanty. 
# It means there is something wrong for that SN. I could 'comment' that SN.

mu_SALT2[1]

array([0.15481551, 0.13274868, 0.12909198, 0.13087797, 0.12688074,
       0.12887832, 0.12976977, 0.16770021, 0.13272181, 0.13059   ,
       0.16879157, 0.14151195, 0.15377446, 0.23097719, 0.24685683,
       0.14514186, 0.13493468, 0.15319522, 0.1998282 , 0.17438752,
       0.24199106, 0.14363616, 0.13575731, 0.16119983, 0.14248966,
       0.13026901, 0.14319062, 0.14340485, 0.14528599, 0.12848738,
       0.14586451, 0.135249  , 0.13649914, 0.15983834, 0.12975733,
       0.14565815, 0.14072648, 0.23068019, 0.13504692, 0.12984852,
       0.1446374 , 0.12950428, 0.2078359 , 0.15885971, 0.12907783,
       0.14641459, 0.14072591, 0.1379126 , 0.14564967, 0.13971411,
       0.44680659, 0.17064811, 0.2383414 , 0.27176105, 0.18075139,
       0.13447164])

In [32]:
# Inspect for negative values for the distance-modulus uncertainty that 
# comes from Tripp formula. It means there is something wrong for that SN.
# I could 'comment' that SN.

mu_SALT2[2]

array([0.00240233, 0.00340655, 0.00311324, 0.00314828, 0.00310821,
       0.00303114, 0.00326272, 0.00453904, 0.00298506, 0.00289957,
       0.00299069, 0.00382378, 0.00322291, 0.00299007, 0.0026823 ,
       0.00364765, 0.00352384, 0.01003466, 0.00308104, 0.00354109,
       0.03829929, 0.00514929, 0.00411679, 0.00346445, 0.00735589,
       0.00403835, 0.00322294, 0.00341041, 0.00386104, 0.00332198,
       0.00347236, 0.00326293, 0.00281009, 0.00312356, 0.00344512,
       0.00592461, 0.00365387, 0.00271559, 0.00343428, 0.00349798,
       0.00310022, 0.00381698, 0.00271399, 0.00820964, 0.00346833,
       0.00319508, 0.00656351, 0.00335296, 0.0031844 , 0.00330866,
       0.00264953, 0.0132323 , 0.00712213, 0.00241101, 0.00316849,
       0.00386521])

In [33]:
# Checking if any supernovae has NEGATIVE value for the distance-modulus
# uncertainty that comes from Tripp formula, or 'nan' in its total uncertanty;
# in any case it means that there is something wrong for that SN. I could 
# 'comment' that SN.

print '# SN name|   mu   |  sigma_mu | sigma^2_mu_photo'
for i in range(len(mu_SALT2[0])):
    print '%-8s  |  %.5f  |  %.5f   | %.5f'%(
        snanaSalt2Fit_np['f1'][i], mu_SALT2[0][i], mu_SALT2[1][i], mu_SALT2[2][i] )
    

# SN name|   mu   |  sigma_mu | sigma^2_mu_photo
1999ee    |  33.21893  |  0.15482   | 0.00240
2000ca    |  35.01750  |  0.13275   | 0.00341
2008ar    |  35.36404  |  0.12909   | 0.00311
2008bf    |  35.15791  |  0.13088   | 0.00315
2008hj    |  36.01749  |  0.12688   | 0.00311
2009aa    |  35.40535  |  0.12888   | 0.00303
2009ad    |  35.39276  |  0.12977   | 0.00326
2009ag    |  33.25474  |  0.16770   | 0.00454
2009cz    |  34.88732  |  0.13272   | 0.00299
2009D     |  35.04688  |  0.13059   | 0.00290
2009Y     |  32.76396  |  0.16879   | 0.00299
2010kg    |  33.89904  |  0.14151   | 0.00382
2011ao    |  33.21391  |  0.15377   | 0.00322
2011B     |  31.66918  |  0.23098   | 0.00299
2011by    |  31.98274  |  0.24686   | 0.00268
2011df    |  33.99352  |  0.14514   | 0.00365
f20080514-002  |  35.18383  |  0.13493   | 0.00352
2001ba    |  35.69114  |  0.15320   | 0.01003
2005cf    |  32.41111  |  0.19983   | 0.00308
2006D     |  33.00760  |  0.17439   | 0.00354
2006lf    |  33.10745  |  

In [13]:
""" 
# SN name|   mu   |  sigma_mu | sigma^2_mu_photo
1999ee    |  33.21893  |  0.15482   | 0.00240
2000ca    |  35.01750  |  0.13275   | 0.00341
2008ar    |  35.36404  |  0.12909   | 0.00311
2008bf    |  35.15791  |  0.13088   | 0.00315
2008hj    |  36.01749  |  0.12688   | 0.00311
2009aa    |  35.40535  |  0.12888   | 0.00303
2009ad    |  35.39276  |  0.12977   | 0.00326
2009ag    |  33.25474  |  0.16770   | 0.00454
2009cz    |  34.88732  |  0.13272   | 0.00299
2009D     |  35.04688  |  0.13059   | 0.00290
2009Y     |  32.76396  |  0.16879   | 0.00299
2010kg    |  33.89904  |  0.14151   | 0.00382
2011ao    |  33.21391  |  0.15377   | 0.00322
2011B     |  31.66918  |  0.23098   | 0.00299
2011by    |  31.98274  |  0.24686   | 0.00268
2011df    |  33.99352  |  0.14514   | 0.00365
f20080514-002  |  35.18383  |  0.13493   | 0.00352
2001ba    |  35.69114  |  0.15320   | 0.01003
2005cf    |  32.41111  |  0.19983   | 0.00308
2006D     |  33.00760  |  0.17439   | 0.00354
2006lf    |  33.10745  |  0.24199   | 0.03830
2008hs    |  34.80596  |  0.14364   | 0.00515
2009al    |  34.99600  |  0.13576   | 0.00412
2009an    |  33.32071  |  0.16120   | 0.00346
2008gb    |  36.15102  |  0.14249   | 0.00736
2009bv    |  36.37652  |  0.13027   | 0.00404
2004eo    |  33.92436  |  0.14319   | 0.00322
2004ey    |  34.12288  |  0.14340   | 0.00341
2005el    |  34.11444  |  0.14529   | 0.00386
2005iq    |  35.97470  |  0.12849   | 0.00332
2005kc    |  33.97212  |  0.14586   | 0.00347
2005ki    |  34.67375  |  0.13525   | 0.00326
2006ax    |  34.41658  |  0.13650   | 0.00281
2006bh    |  33.36583  |  0.15984   | 0.00312
2006bt    |  35.74917  |  0.12976   | 0.00345
2006kf    |  34.80471  |  0.14566   | 0.00592
2007A     |  34.52997  |  0.14073   | 0.00365
2007af    |  32.07550  |  0.23068   | 0.00272
2007bc    |  34.66329  |  0.13505   | 0.00343
2007bd    |  35.63540  |  0.12985   | 0.00350
2007ca    |  34.41652  |  0.14464   | 0.00310
2007jg    |  36.26046  |  0.12950   | 0.00382
2007le    |  32.22046  |  0.20784   | 0.00271
2008bc    |  33.99679  |  0.15886   | 0.00821
2008gp    |  35.68398  |  0.12908   | 0.00347
2008hv    |  33.80562  |  0.14641   | 0.00320
2007ai    |  35.79065  |  0.14073   | 0.00656
2007as    |  35.02262  |  0.13791   | 0.00335
2001bt    |  33.75528  |  0.14565   | 0.00318
2001cz    |  34.02668  |  0.13971   | 0.00331
1998bu    |  30.29529  |  0.44681   | 0.00265
1999ek    |  34.26677  |  0.17065   | 0.01323
2000E     |  31.57443  |  0.23834   | 0.00712
2001el    |  31.53889  |  0.27176   | 0.00241
2002dj    |  32.89399  |  0.18075   | 0.00317
2010ai    |  35.13503  |  0.13447   | 0.00387
"""
0

0

# Determine the mean absolute magnitude

In [34]:
# Define the average in the absolute value of the residual
# These functions are going to be minimized.

# AbsResidual values
res_mu_np_int = mu_SALT2[0] - DistanceMuVector(z_np, OmMFix, wFix, HoFix)

# Inverse-variance weighted average function to be minimized 
def WeightedAbsResidual_fun(deltaMo, UpLimit, LowLimit, AbsResidual_Roof):
    residuals_int = res_mu_np_int + deltaMo
    WeightedAbsResidual_int = np.average(residuals_int, weights=mu_SALT2[1] )
    
    if deltaMo < UpLimit and deltaMo > LowLimit:  
        # For some unknown reason, simplex is search the maximum instead of
        # the minimimum, so I had to define the average absolute mag as 1/WeightedAbsResidual
        # WeightedAbsResidual_final = 1/WeightedAbsResidual_int
        WeightedAbsResidual_final = WeightedAbsResidual_int
        
    else: WeightedAbsResidual_final = AbsResidual_Roof
        
    return abs(WeightedAbsResidual_final)

#-----------------------------------------------------

# Simple average function to be minimized 
def AverageAbsResidual_fun(deltaMo, UpLimit, LowLimit, AbsResidual_Roof):
    residuals_int = res_mu_np_int + deltaMo
    # AverageAbsResidual_int = np.mean(residuals_int)
    AverageAbsResidual_int = np.sum(residuals_int)/len(residuals_int)
     
    if deltaMo < UpLimit and deltaMo > LowLimit: 
        # For some unknown reason, simplex is search the maximum instead of
        # the minimimum, so I had to define the average absolute mag as 1/AverageAbsResidual_int
        # AverageAbsResidual_final = 1/AverageAbsResidual_int
        AverageAbsResidual_final = AverageAbsResidual_int
        
    else: AverageAbsResidual_final = AbsResidual_Roof

    return abs(AverageAbsResidual_int)

In [35]:
print '# Tests:'
print '#', WeightedAbsResidual_fun(-2.3, 3, -3, 1)
print '#', AverageAbsResidual_fun(-0.0153, 0.2, -0.2, 1)

# Tests:
# 2.22003154831089
# 0.06383338274987903

# Tests:
# 2.22003154831089
# 0.06383338274987903


In [37]:
# Determine the value of the constant deltaMo in order to obtain a
# weighted average value of zero in the Hubble residual

import scipy.optimize

# Search limits:
# UpLimit = 0.2; LowLimit = -0.2
UpLimit = 3; LowLimit = -3

# Assume this value for deltaMo in case of the search is outside the range
# indicated above.
Residual_Roof = 10 

WeightedAbsResidual_Out = scipy.optimize.minimize_scalar(WeightedAbsResidual_fun, 
                                        args=(UpLimit, LowLimit, Residual_Roof))

AverageAbsResidual_Out = scipy.optimize.minimize_scalar(AverageAbsResidual_fun, 
                                        args=(UpLimit, LowLimit, Residual_Roof))

# Redefining the values:
SimplexResult_1 = [WeightedAbsResidual_Out['x'] ]
SimplexResult_2 = [AverageAbsResidual_Out['x'] ]

print WeightedAbsResidual_Out
print '#', SimplexResult_1, ' = value of deltaMo that minimize the Hubble residual.'
print 

print AverageAbsResidual_Out
print '#', SimplexResult_2, ' = value of deltaMo that minimize the Hubble residual.'

     fun: 2.2233628330899783e-10
    nfev: 37
     nit: 36
 success: True
       x: -0.0799684519114457
# [-0.0799684519114457]  = value of deltaMo that minimize the Hubble residual.

     fun: 1.8231054959334207e-10
    nfev: 31
     nit: 30
 success: True
       x: -0.07913338256756848
# [-0.07913338256756848]  = value of deltaMo that minimize the Hubble residual.


In [38]:
"""
     fun: 6.2205205415565593e-12
    nfev: 28
     nit: 27
 success: True
       x: 0.0045658128791809327
# [0.0045658128791809327]  = value of deltaMo that minimize the Hubble residual.

     fun: 6.6409884697359133e-12
    nfev: 30
     nit: 29
 success: True
       x: 0.003668901213883702
# [0.003668901213883702]  = value of deltaMo that minimize the Hubble residual.
"""
0

0

In [39]:
# Verifying the minimim using a different python function to find the minimum.

# Determine the value of the constant deltaMo in order to obtain a
# weighted average value of zero in the Hubble residual

from scipy.optimize import fmin

# Search limits:
# UpLimit_test = 0.1; LowLimit_test = -0.1
UpLimit_test = 3; LowLimit_test = -3

# Assume this value for deltaMo in case of the search is outside the range
# indicated above.
Residual_Roof_test = 10

InitialGuess_test = -0.0423

# Find the value of deltaMo that minimizes the averager functions 
SimplexResult_1_test = fmin(WeightedAbsResidual_fun, InitialGuess_test, # xtol=1e-10, ftol=1e-10, 
                          # retall=True,
                          args=(UpLimit_test, LowLimit_test, Residual_Roof_test) )
print SimplexResult_1_test, ' = value of deltaMo that minimize the Hubble residual.'
print 

SimplexResult_2_test = fmin(AverageAbsResidual_fun, InitialGuess_test,  # xtol=1e-10, ftol=1e-10, 
                          # retall=True,
                          args=(UpLimit_test, LowLimit_test, Residual_Roof_test) )
print SimplexResult_2_test, ' = value of deltaMo that minimize the Hubble residual.'

Optimization terminated successfully.
         Current function value: 0.000005
         Iterations: 12
         Function evaluations: 24
[-0.07997344]  = value of deltaMo that minimize the Hubble residual.

Optimization terminated successfully.
         Current function value: 0.000019
         Iterations: 12
         Function evaluations: 24
[-0.07911422]  = value of deltaMo that minimize the Hubble residual.


In [40]:
# Print results

print '# deltaMo =', SimplexResult_1, ', for a weighted average abs residual of:', \
WeightedAbsResidual_fun(SimplexResult_1[0], UpLimit, LowLimit, Residual_Roof)  

print '# deltaMo =', SimplexResult_2, ', for an average abs residual of:', \
AverageAbsResidual_fun(SimplexResult_2[0], UpLimit, LowLimit, Residual_Roof)    

# deltaMo = [-0.0799684519114457] , for a weighted average abs residual of: 2.2233628330899783e-10
# deltaMo = [-0.07913338256756848] , for an average abs residual of: 1.8231054959334207e-10


In [41]:
# deltaMo min = [-0.03932578] , for a weighted average abs residual of: 3.21942477339e-05
# deltaMo min = [-0.03502969] , for an average abs residual of: 8.94368852025e-06

### Compute final residual values and checking the Hubble residual

In [42]:
# Inverse-variance weighted average

res_mu_weight_np = (mu_SALT2[0] - DistanceMuVector(z_np, OmMFix, wFix, HoFix) 
                    +SimplexResult_1[0])  
WeightedResidual = np.average(res_mu_weight_np, weights=mu_SALT2[1] )

print '# NOTE: THE FOLLOWING VALUE SHOULD BE VERY SMALL (< 1e-5), OTHERWISE, IT S WRONG.'
print '#', WeightedResidual
# 3.21942477339e-05

# NOTE: THE FOLLOWING VALUE SHOULD BE VERY SMALL (< 1e-5), OTHERWISE, IT S WRONG.
# -2.2233628330899783e-10


In [43]:
# Simple average

res_mu_mean_np = (mu_SALT2[0] - DistanceMuVector(z_np, OmMFix, wFix, HoFix) 
                  +SimplexResult_2[0])  

print '# NOTE: THE FOLLOWING VALUE SHOULD BE VERY SMALL (< 1e-5), OTHERWISE, IT S WRONG.'
print '#', np.mean(res_mu_mean_np)
# NOTE: THE FOLLOWING VALUE SHOULD BE VERY SMALL (~ 1e-10), OTHERWISE, IT S WRONG.
# 8.94368852025e-06

# NOTE: THE FOLLOWING VALUE SHOULD BE VERY SMALL (< 1e-5), OTHERWISE, IT S WRONG.
# 1.8231054959334207e-10


------

# Write the results to a text files

### My format

To be read by '11_DistanceMu_HubbleDiagram_v2_17.ipynb'

In [44]:
# Write down to a text file the results using the same datatable 
# format output than my 11_DistanceMu_HubbleDiagram_v2_17.ipynb code.

textfile_1 = open(DirSaveOutput+'DistanceMu_Good_AfterCutoffs_Main_.txt','w')
textfile_2 = open(DirSaveOutput+'DistanceMu_fromSALT2.txt','w') # To share with collaborators

now = datetime.datetime.now() # Read the time and date right now
text_timenow = now.strftime("%Y-%m-%d (yyyy-mm-dd); %H:%M hrs.")
text_Author = '# Data table created by: Arturo Avelino \n'
text_Date   = '# On date: %s \n'%text_timenow
text_script = '# Script used: %s \n'%NotebookName
text_line = '#'+'-'*70 + '\n'

text_01 = '# Distance modulus determined from the combination of SALT2.JLA-B14 output fit \n\
# parameters and the Tripp formula, using the global parameters reported by Scolnic etal. 2017. \n'
text_02 = '# Values of the global parameters used: \n'
text_03 = '# alpha = %s | beta = %s |  sigmaInt = %s \n'%(alpha,beta,sigmaInt)
text_04 = '# M = %s. Assumed value for the absolute magnitude in Tripp \
formula. \n'%MoFix
text_05 = '# Delta_M = %s. Additional value to the absolute magnitude in Tripp formula \n\
# to have a zero-value weighted average in the Hubble residual plot. \n'%SimplexResult_1[0]
text_06 = '# Om_matter = %s | w = %s | Ho = %s km/s/Mpc | Flat. \
Cosmological parameters assumed. \n'%(OmMFix, wFix, HoFix)
text_07 = '# %s km/s: Peculiar velocity uncertainty assumed.  \n'%sigma_vPec

text_08_1 = '# SN name     z_CMB   error_zcmb   mu     error_mu   mu_residual chi2_dof  \
Sample  appMag_TBmax  error_appMagTBmax  mu_LCDM     sigma_muLCDM_vPec AbsMagTBmax \
error_AbsMagTBmax  TBmax       Error_TBmax    PhaseB        zhelio        err_zhelio   \
dm15         error_dm15      EBVhost      error_EBVhost   EBV_MW        error_EBV_MW     \
Alamb      err_Alamb      R_F           mu_Snoopy     error_muSnoopy   TBmax           \
Error_TBmax  appMag_TBmax  error_appMagTBmax  Notes  \n'

text_08_2 = '# SN name     z_CMB   error_zcmb   mu     error_mu   \
appMag_TBmax  error_appMagTBmax  mu_LCDM     sigma_muLCDM_vPec \n'


textfile_1.write(text_01); textfile_1.write(text_02); textfile_1.write(text_03);
textfile_1.write(text_04); textfile_1.write(text_05); textfile_1.write(text_06);
textfile_1.write(text_07); 
textfile_1.write(text_line)
textfile_1.write(text_Author); textfile_1.write(text_Date); textfile_1.write(text_script); 
textfile_1.write(text_line)
textfile_1.write(text_08_1)  

textfile_2.write(text_01); textfile_2.write(text_02); textfile_2.write(text_03); 
textfile_2.write(text_04); textfile_2.write(text_05); textfile_2.write(text_06);
textfile_2.write(text_07); 
textfile_2.write(text_line)
textfile_2.write(text_Author); textfile_2.write(text_Date); textfile_2.write(text_script); 
textfile_2.write(text_line)
textfile_2.write(text_08_2)  

#------------------------------

countSN = 0
for i in range(len(snanaSalt2Fit_np)):
    
    snname = snanaSalt2Fit_np['f1'][i]
    zz_int = z_np[i]
    e_zz_int = zerr_np[i]
    muLCDM_int = DistanceMu(zz_int, OmMFix, wFix, HoFix)
    sigma_muLCDM_vPec_int = np.sqrt(sigma2_pec(zz_int, e_zz_int, sigma_vPec))
    chi2dof_int = chi2_np[i]/Ndof_np[i]
    mu_final = mu_SALT2[0][i] +SimplexResult_1[0]
    muerr_final = mu_SALT2[1][i]
    muResidual_int = res_mu_weight_np[i]
    mb_int = mb_np[i]
    mberr_int = mberr_np[i]
    
    #-------
    """  
    # OLD
    zz_int = snanaSalt2Fit_np['f2'][i]
    e_zz_int = snanaSalt2Fit_np['f3'][i]
    muLCDM_int = DistanceMu(zz_int, OmMFix, wFix, HoFix)
    sigma_muLCDM_vPec_int = np.sqrt(sigma2_pec(zz_int, e_zz_int, sigma_vPec))
    chi2dof_int = snanaSalt2Fit_np['f17'][i]/snanaSalt2Fit_np['f18'][i]
    mu_final = mu_SALT2[0][i] +SimplexResult_1[0] """
    
    textfile_1.write('sn%-10s  %.5f  %.5f  \
%.5f  %.5f  %10.6f  \
%10.6f    %s     \
%.4f       %.4f            \
%.8f  %.8f   \
0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 \n'%(  
        snname, zz_int, e_zz_int,
        mu_final, muerr_final, muResidual_int, 
        chi2dof_int, SampleFlag, 
        mb_int, mberr_int,
        muLCDM_int, sigma_muLCDM_vPec_int) )   
    
    #-------
    
    textfile_2.write('sn%-10s  %.5f  %.5f  \
%.5f  %.5f     \
%.4f       %.4f            \
%.8f  %.8f  \n'%(  
        snname, zz_int, e_zz_int,
        mu_final, muerr_final,
        mb_int, mberr_int,
        muLCDM_int, sigma_muLCDM_vPec_int) )  
    
    countSN = countSN + 1
    
    
text_12 = '# %s SNeIa in this list.'%countSN
textfile_2.write(text_line)
textfile_1.write(text_12); textfile_2.write(text_12);
textfile_1.close(); textfile_2.close()

#### SNANA-SALT2mu.exe format

This doesn't work yet for the case of reading David Jones fitres file.

In [23]:
# Write down to a text file the results using the same datatable 
# format output than SALT2mu.exe.

if FitresFormat == 'v10_35g': 
    textfile_1 = open(DirSaveOutput+'DistanceMu_SALT2muFormat_.txt','w')

    now = datetime.datetime.now() # Read the time and date right now
    text_timenow = now.strftime("%Y-%m-%d (yyyy/mm/dd); %H:%M hrs.")
    text_line = '#'+'-'*80 + '\n'

    #--- Copy/paste of the cell above -----
    textfile_1.write('# Distance modulus determined from the combination of SALT2.JLA-B14 \
    output fit parameters and the Tripp formula, using the global parameters reported \
    by Scolnic etal. 2017. \n')
    textfile_1.write('# Values of the global parameters used: \n')
    textfile_1.write('# alpha = %s | beta = %s |  sigmaInt = %s \n'%(alpha,beta,sigmaInt))
    textfile_1.write('# M = %s. Assumed value for the absolute magnitude in Tripp \
    formula. \n'%MoFix)
    textfile_1.write('# Delta_M = %s. Additional value to the absolute magnitude in Tripp \
    formula to have a zero-value weighted average in the Hubble residual plot. \n'%SimplexResult_1[0])
    textfile_1.write('# Om_matter = %s | w = %s | Ho = %s km/s/Mpc | Flat. \
    Cosmological parameters assumed. \n'%(OmMFix, wFix, HoFix))
    textfile_1.write('# %s km/s: Peculiar velocity uncertainty assumed.  \n'%sigma_vPec)
    textfile_1.write(text_line)

    textfile_1.write('# Data table created by: Arturo Avelino \n')
    textfile_1.write('# On date: %s \n'%text_timenow)
    textfile_1.write('# Script used: %s \n'%NotebookName)
    textfile_1.write(text_line)

    #------- end copy/paste -------------

    textfile_1.write('# SN name       z        zERR     x0           x0ERR       c       cERR     \
    x1      x1ERR   PKMJD   PKMJDERR mB      mBERR    COVx0x1     COVx0c      COVx1c     CHI2   NDOF  \
    FITPROB      SNRMAX1    SNRMAX2    SNRMAX3 SURVEY TYPE   MU      MUERR    MU_residual   \n')

    for i in range(len(snanaSalt2Fit_np)):

        mu_final = mu_SALT2[0][i] +SimplexResult_1[0]

        textfile_1.write('SN: %-10s  %.5f  %.5f  %.5e  %.3e  %7.4f  %.4f  %7.4f  %.4f  %.3f  %.2f \
    %.4f  %.4f  %10.3e  %10.3e  %10.3e  %5.1f  %4.0f  %.3e  %9.3f  %9.3f  %9.3f  %2.0f     %2.0f  \
    %.5f  %.5f  %10.6f   \n'%(
            snanaSalt2Fit_np['f1'][i], snanaSalt2Fit_np['f2'][i], snanaSalt2Fit_np['f3'][i],
            snanaSalt2Fit_np['f4'][i], snanaSalt2Fit_np['f5'][i], snanaSalt2Fit_np['f6'][i],
            snanaSalt2Fit_np['f7'][i], snanaSalt2Fit_np['f8'][i], snanaSalt2Fit_np['f9'][i],
            snanaSalt2Fit_np['f10'][i], snanaSalt2Fit_np['f11'][i],
            snanaSalt2Fit_np['f12'][i], snanaSalt2Fit_np['f13'][i],
            snanaSalt2Fit_np['f14'][i], snanaSalt2Fit_np['f15'][i], snanaSalt2Fit_np['f16'][i],
            snanaSalt2Fit_np['f17'][i], snanaSalt2Fit_np['f18'][i], snanaSalt2Fit_np['f19'][i],
            snanaSalt2Fit_np['f20'][i], snanaSalt2Fit_np['f21'][i], snanaSalt2Fit_np['f22'][i],
            snanaSalt2Fit_np['f23'][i], snanaSalt2Fit_np['f24'][i],
            mu_final, mu_SALT2[1][i], res_mu_weight_np[i]  ))

    textfile_1.close()

---------

# Hubble diagram plot

In [45]:
# USER

fontsizePlot = 12

In [46]:
#    Theoretical values
# Array plot the -theoretical- (spectroscopic) distance modulus 
nbins1= 51
z1 = np.linspace(min(z_np)-0.001, max(z_np)+0.001, nbins1)
mu0 = DistanceMuVector(z1, OmMFix, wFix, HoFix)

#----------------------------------------

# PLOTTING

plt.errorbar(z_np, mu_SALT2[0]+SimplexResult_1[0], yerr=mu_SALT2[1], fmt='.')
plt.plot(z1, mu0)

plt.grid(True)
plt.xlabel('Redshift', fontsize=fontsizePlot)
plt.ylabel(r'Distance modulus $\mu$', fontsize=fontsizePlot)
plt.title('SALT2 Hubble diagram, %s sample'%sample)
plt.tight_layout()

plt.savefig(DirSaveOutput+'Plot_Hubble_SALT2_.png')
plt.close()

# Hubble residual plot

In [47]:
#    Theoretical values
# Array plot the -theoretical- (spectroscopic) distance modulus 
nbins1= 51
z1 = np.linspace(min(z_np)-0.001, max(z_np)+0.001, nbins1)
mu1 = np.zeros(len(z1))

#----------------------------------------

# res_mu_np_int
# res_mu_weight_np

plt.errorbar(z_np, res_mu_weight_np, yerr=mu_SALT2[1], fmt='.')


plt.plot(z1, mu1)
# plt.text(min(z_np)+0.01, min(res_mu_weight_np), r'$\sigma_{\rm pec}$ = %s km/s'%sigma_vPec)
plt.grid(True)
plt.xlabel('Redshift', fontsize=fontsizePlot)
plt.ylabel(r'$\mu - \mu_{\rm \Lambda CDM}$', fontsize=fontsizePlot+3) 
plt.title('SALT2 Hubble residual, %s sample'%sample)
plt.tight_layout()

plt.savefig(DirSaveOutput+'Plot_HubbleRes_SALT2_.png')
plt.close()